In [21]:
pip install matplotlib

In [23]:
pip install seaborn

In [25]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install mysql-connector-python


In [29]:
import pandas as pd
import mysql.connector
import os
csv_files=[('customers.csv','customers'),('orders.csv','orders'),('sellers.csv','sellers'),('products.csv','products'),('geolocation.csv', 'geolocation')
           ,('payments.csv','payments'),('order_items.csv','order_items')]
conn=mysql.connector.connect(
    host='localhost',
    user='root',
    password='Yogita@963',
    database='ecommerce'
)
cursor = conn.cursor()
folder_path = 'C:/Users/Radha/Desktop/py+sql'
def sql_dtype(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return'DATETIME'
    else:
        return'TEXT'
for csv_file,table_name in csv_files:
    file_path=os.path.join(folder_path,csv_file)
    df=pd.read_csv(file_path)
    df=df.where(pd.notnull(df),None)
    print(f"Processing{csv_file}")
    print(f"NaN values before replacement:\n{df.isnull().sum()}\n")
    df.columns=[col.replace(' ','_').replace('-','_').replace('.','_') for col in df.columns]
    columns = ', '.join([f'`{col}` {sql_dtype(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS `{table_name}` ({columns})'

    cursor.execute(create_table_query)
    
    for _, row in df.iterrows():
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO `{table_name}` ({', '.join(['`' + col + '`' for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        cursor.execute(sql, values)
    conn.commit()
conn.close()



Processingcustomers.csv
NaN values before replacement:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Processingorders.csv
NaN values before replacement:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Processingsellers.csv
NaN values before replacement:
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

Processingproducts.csv
NaN values before replacement:
product_id                      0
product category              610
product_name_length           610
product_description_length    610
product_photos_qty            610
product_

In [11]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
    --------------------------------------- 0.3/16.4 MB ? eta -:--:--
   --- ------------------------------------ 1.3/16.4 MB 3.9 MB/s eta 0:00:04
   ----- ---------------------------------- 2.4/16.4 MB 4.3 MB/s eta 0:00:04
   -------- ------------------------------- 3.4/16.4 MB 4.5 MB/s eta 0:00:03
   ---------- ----------------------------- 4.5/16.4 MB 4.5 MB/s eta 0:00:03
   ------------ --------------------------- 5.0/16.4 MB 4.2 MB/s eta 0:00:03
   -------------- ------------------------- 6.0/16.4 MB 4.3 MB/s eta 0:00:03
   ----------------- ---------------------- 7.3/16.4 MB 4.5 MB/s eta 0:00:02
   -------------------- ------------------- 8.4/16.4 MB 4.6 MB/s eta 0:00:02
   ----------------------- ---------------- 9.4/16.4 MB 4.7 MB/s eta 0:00:02
   ------------------------ --------------- 10.2/16.4 MB 4.7 MB/s eta 0:00:02
   -------------------------- ------------- 11.0/16.4 MB 4.4 MB/s eta 0:00:02
   --------

In [13]:
pip install pandas 

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import mysql.connector
db=mysql.connector.connect(host="localhost",
                           username="root",
                           password="Yogita@963",
                           database="ecommerce")
cur=db.cursor()


# List all unique cities where customers are located.

In [49]:
query="select distinct customer_city from customers"
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["Cities"])
df.index=df.index+1
df.head(10)

,Cities
1,franca
2,sao bernardo do campo
3,sao paulo
4,mogi das cruzes
5,campinas
6,jaragua do sul
7,timoteo
8,curitiba
9,belo horizonte
10,montes claros


# Count the number of orders placed in 2017.

In [9]:
query="select count(order_id) from orders where year(order_purchase_timestamp)=2017 "
cur.execute(query)
data=cur.fetchall()
data[0][0]

45101

# Find the total sales per category

In [40]:
query="""select products.product_category,round(sum(payments.payment_value),2)
from order_items
inner join payments
on payments.order_id=order_items.order_id
inner join products
on products.product_id=order_items.product_id
group by product_category"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["Category","Sales"])
df=df.sort_values(by="Sales",ascending=False)
df=df.reset_index(drop=True)
df.index=df.index+1
df.head()

,Category,Sales
1,bed table bath,1712553.67
2,HEALTH BEAUTY,1657373.12
3,computer accessories,1585330.45
4,Furniture Decoration,1430176.39
5,Watches present,1429216.68


# Calculate the percentage of orders that were paid in installments.

In [36]:
query="""select (sum(case when payment_installments>=1 then 1 else 0 end))/count(*)*100 from payments"""
cur.execute(query)
data=cur.fetchall()
data[0][0]

Decimal('99.9981')

#  Count the number of customers from each state

In [39]:
query="""select customer_state,count(customer_id) from customers group by 1 """
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["State","Customers"])
df=df.sort_values(by="Customers",ascending=False)
df=df.reset_index(drop=True)
df.index=df.index+1
df

,State,Customers
1,SP,41746
2,RJ,12852
3,MG,11635
4,RS,5466
5,PR,5045
6,SC,3637
7,BA,3380
8,DF,2140
9,ES,2033
10,GO,2020


#  Calculate the number of orders per month in 2018.

In [68]:
query="""select monthname(order_purchase_timestamp),count(order_id) from orders where year(order_purchase_timestamp)=2018
group by 1 """
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["Months","Orders"])
df=df.sort_values(by="Orders",ascending=False)
df=df.reset_index(drop=True)
df.index=df.index+1
df

,Months,Orders
1,January,7269
2,March,7211
3,April,6939
4,May,6873
5,February,6728
6,August,6512
7,July,6292
8,June,6167
9,September,16
10,October,4


#  Calculate the Percentage of total revenue contributed by each product category

In [34]:
query="""select upper(products.product_category),round((sum(payments.payment_value)/(select sum(payment_value) from payments))*100 ,2)
from products join order_items
on products.product_id=order_items.product_id
join payments
on payments.order_id=order_items.order_id
group by products.product_category
order by 2 desc"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["Product_Category","Revenue%"])
df.index=df.index+1
df.head(10)

,Product_Category,Revenue%
1,BED TABLE BATH,10.70
2,HEALTH BEAUTY,10.35
3,COMPUTER ACCESSORIES,9.90
4,FURNITURE DECORATION,8.93
5,WATCHES PRESENT,8.93
6,SPORT LEISURE,8.70
7,HOUSEWARES,6.84
8,AUTOMOTIVE,5.32
9,GARDEN TOOLS,5.24
10,COOL STUFF,4.87


#  Identify the correlation between product price and the number of times a product has been purchased.

In [42]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import mysql.connector

db=mysql.connector.connect(host="localhost",
                           username="root",
                           password="Yogita@963",
                           database="ecommerce")
cur=db.cursor()
query="""
select products.product_category,count(order_items.product_id) as order_count,round(avg(price),2) as price from products join order_items
on products.product_id=order_items.product_id
group by 1"""
cur.execute(query)
data=cur.fetchall()
df=pd.DataFrame(data,columns=["Product_id","order_count","price"])
df.index=df.index+1
correlation=df[['order_count','price']].corr()
print( correlation)


             order_count     price
order_count     1.000000 -0.106315
price          -0.106315  1.000000
